
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
print(ipa.__version__)
print(toyplot.__version__)

0.9.40
0.18.0


In [3]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [4]:
# the path to your HDF5 formatted snps file
data = "./H_st_11rm.snps.hdf5"

In [5]:
imap = {
#"ref": ["reference"],
#"Inam": [],
#"Puru": ["H_pe_A24346_pu"],
"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
#"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
#"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
#"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [6]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="H_st_11rm_east",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 21
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 18332
Filtered (mincov): 1625545
Filtered (minmap): 1683639
Filtered (combined): 1683930
Sites after filtering: 18731
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [7]:
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_east-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_east-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_east-<built-in function abs>-1.strfile.txt')

In [8]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000

In [9]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True,)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[#################   ]  85% 1:16:13 | running 70 structure jobs 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
struct = ipa.structure(
    data=data, 
    name="H_st_11rm_east", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [H_st_11rm_east]


In [13]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-5422.23,217.421,0.00,0.00
3,10,2.105,-5554.93,223.754,-132.70,471.10
4,10,2.027,-5216.53,339.655,338.40,688.48
5,10,0.009,-5566.61,872.041,-350.08,7.62
6,10,0.419,-5924.31,893.784,-357.70,374.83
7,10,0.007,-5907.18,627.277,17.13,4.67
8,10,0.000,-5885.38,567.326,21.80,0.00


In [14]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8db1129ef343428a904b2de855e788d1" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 5250 5500 5750 6000 estLnProbMean 0 1 2 3 deltaK

In [15]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [16]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [17]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t65ff1de8fb144977b70cc8d9e808dd8d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_oc_A311_jigu H_oc_T355_jigu H_oc_T15847_jigu 0.0 0.5 1.0

In [18]:
k = 3
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K3] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t10ae7a6db0304f45be7faf3aaff2a308" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa 0.0 0.5 1.0

In [20]:
k = 4
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K4] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="td2660bbf36734d27b1557a57bfbce165" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa 0.0 0.5 1.0

In [21]:
k = 5
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K5] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t2fc468bbe0c04d45860c9f8bfc93470e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa 0.0 0.5 1.0

In [22]:
k = 6
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K6] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="te06535c187dc4c9a98d2723c8e1dc061" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa 0.0 0.5 1.0

# PCA

In [23]:
#!conda install scikit-learn -c conda-forge -y

In [24]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [25]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 21
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 18332
Filtered (mincov): 1054500
Filtered (minmap): 1683639
Filtered (combined): 1683930
Sites after filtering: 18731
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [26]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 2038/18731


In [27]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b23ba8ac1d0>,
 <toyplot.mark.Point at 0x2b23ba43b090>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t9270890220e742ec873aedbed1d4fbf5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -5 0 5 10 PC0 (29.3%) explained -5 0 5 10 PC1 (16.2%) explained JiGu Para Mach

In [28]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("H_st_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
H_oc_A311_jigu,4.63,11.83,-1.94,0.74,-0.17,1.17,0.05,-0.51,0.06,0.47
H_oc_T15847_jigu,4.40,11.75,-2.35,0.78,0.29,-1.02,-0.01,0.43,-0.04,-0.59
H_oc_T355_jigu,-4.35,-0.72,0.15,0.04,-3.35,0.51,-0.58,1.73,-1.83,-1.15
H_ro_A547_ma,-5.60,-1.08,-0.22,3.27,4.06,-0.43,-0.52,0.76,-3.20,2.45
H_ro_J213_ma,-4.99,-0.72,0.55,2.30,-0.23,-1.12,0.03,-2.43,4.29,-1.54
H_ro_J305_ma,-5.01,-0.24,-0.41,-1.31,0.26,0.81,0.33,-0.35,-1.62,-1.37
H_ro_J508_ma,-4.17,-0.14,-0.35,-2.27,3.54,-0.66,-0.48,-0.59,2.26,3.69
H_ro_J774_ma,-4.24,-0.33,-0.70,-1.22,0.40,0.23,1.08,-0.40,0.13,-3.22
H_ro_J775_ma,-4.40,-0.32,-0.88,-3.85,1.49,-1.28,0.85,4.30,-0.56,-0.68
H_ro_J796_ma,-5.41,-1.67,0.45,4.75,2.07,0.12,0.59,-1.80,0.17,-1.54


# Subsampling with replication

In [29]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 2038/18731


<svg class="toyplot-canvas-Canvas" height="300.0px" id="tac4b1a457de648eeaf902861d1cdae15" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -5 0 5 10 PC0 (29.7%) explained -5 0 5 10 15 PC1 (16.5%) explained JiGu Para Mach

# Kmeans imputation (integer)

In [30]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Subsampling SNPs: 2038/18731


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t19cd344eb47d4432a28eb4dce08aef8b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -5 0 5 10 PC0 (29.1%) explained -5 0 5 10 15 PC1 (16.7%) explained JiGu Para Mach

In [31]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [32]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [39]:
pca.run_tsne(subsample=True, perplexity=4.0, n_iter=100000, seed=123)

Subsampling SNPs: 2038/18731


In [40]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t438f0b9d38b547719cc9db82de24b3f7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_oc_A311_jigu H_oc_T15847_jigu H_oc_T355_jigu H_ro_A547_ma H_ro_J213_ma H_ro_J305_ma H_ro_J508_ma H_ro_J774_ma H_ro_J775_ma H_ro_J796_ma H_ro_T1842_ma H_ro_T366_ma H_ro_T385_ma H_ro_T471_ma H_st_A11597_pa H_st_A15208_pa H_st_A15210_pa H_st_A16571_pa H_st_A7597_pa H_st_T16744_pa H_st_T17858_pa -1000 -500 0 500 TNSE component 1 -300 0 300 600 TNSE component 2 JiGu Para Mach

In [41]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_TSNE_perp6.pdf")

# GENETIC DISTNANCES

In [45]:
import ipyrad.analysis as ipa
import toyplot

In [46]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
    
)
dist.run()

Samples: 21
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 18332
Filtered (mincov): 1054500
Filtered (minmap): 1683639
Filtered (combined): 1683930
Sites after filtering: 18731
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


In [47]:
# save to a CSV file
dist.dists.to_csv("H_st_distances.csv")

In [48]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "H_st_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [49]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [50]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="t2ec6811cadf34621b842c8d9bad830be" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.054829 0.054348 0.057391 0.056644 0.058459 0.055843 0.062090 0.059420 0.053868 0.060488 0.151193 0.145694 0.143612 0.142011 0.138113 0.134109 0.134216 0.178955 0.053067 0.178100 H_ro_A547_ma 1 0.054829 0.000000 0.054188 0.055203 0.053601 0.057872 0.051519 0.058940 0.055950 0.054348 0.057338 0.146442 0.142117 0.139395 0.137579 0.134643 0.129037 0.131600 0.170039 0.053120 0.169719 H_ro_J213_ma 2 0.054348 0.054188 0.000000 0.048956 0.050451 0.050558 0.052213 0.054295 0.053227 0.052266 0.054402 0.146495 0.137793 0.140836 0.133789 0.136085 0.132508 0.126635 0.165287 0.053067 0.164220 H_ro_J305_ma 3 0.057391 0.055203 0.048956 0.000000 0.050932 0.049864 0.054936 0.057125 0.052854 0.057979 0.053174 0.147403 0.139234 0.143879 0.135230 0.137099 0.134163 0.129785 0.169825 0.054402 0.168010 H_ro_J508_ma 4 0.056644 0.053601 0.050451 0.050932 0.000000 0.050505 0.054829 0.053601 0.053601 0.056164 0.054028 0.147189 0.139341 0.142598 0.135124 0.138273 0.134483 0.129678 0.167263 0.053120 0.167156 H_ro_J774_ma 5 0.058459 0.057872 0.050558 0.049864 0.050505 0.000000 0.056110 0.053281 0.054455 0.056804 0.053601 0.148470 0.139021 0.142598 0.137259 0.138380 0.134910 0.130212 0.168224 0.054509 0.168224 H_ro_J775_ma 6 0.055843 0.051519 0.052213 0.054936 0.054829 0.056110 0.000000 0.061449 0.057605 0.052907 0.061983 0.147136 0.144306 0.141690 0.138807 0.137473 0.133255 0.134430 0.171587 0.053067 0.170519 H_ro_J796_ma 7 0.062090 0.058940 0.054295 0.057125 0.053601 0.053281 0.061449 0.000000 0.056804 0.058940 0.057765 0.147616 0.138807 0.142171 0.134590 0.137633 0.134590 0.130746 0.168117 0.058139 0.168758 H_ro_T1842_ma 8 0.059420 0.055950 0.053227 0.052854 0.053601 0.054455 0.057605 0.056804 0.000000 0.053494 0.053921 0.146976 0.137206 0.139608 0.132454 0.134643 0.131280 0.125941 0.166729 0.050771 0.166195 H_ro_T366_ma 9 0.053868 0.054348 0.052266 0.057979 0.056164 0.056804 0.052907 0.058940 0.053494 0.000000 0.055096 0.144413 0.140089 0.137473 0.134696 0.132187 0.129465 0.127756 0.167049 0.049704 0.167476 H_ro_T385_ma 10 0.060488 0.057338 0.054402 0.053174 0.054028 0.053601 0.061983 0.057765 0.053921 0.055096 0.000000 0.150713 0.140836 0.145054 0.137900 0.139768 0.133842 0.132241 0.168437 0.053334 0.168331 H_ro_T471_ma 11 0.151193 0.146442 0.146495 0.147403 0.147189 0.148470 0.147136 0.147616 0.146976 0.144413 0.150713 0.000000 0.147509 0.146602 0.141583 0.140142 0.140302 0.137526 0.167316 0.142651 0.164327 H_st_A16571_pa 12 0.145694 0.142117 0.137793 0.139234 0.139341 0.139021 0.144306 0.138807 0.137206 0.140089 0.140836 0.147509 0.000000 0.085687 0.072874 0.075703 0.074796 0.071486 0.171747 0.137900 0.172068 H_st_A7597_pa 13 0.143612 0.139395 0.140836 0.143879 0.142598 0.142598 0.141690 0.142171 0.139608 0.137473 0.145054 0.146602 0.085687 0.000000 0.081256 0.078213 0.076558 0.077091 0.178634 0.137206 0.177887 H_st_T16744_pa 14 0.142011 0.137579 0.133789 0.135230 0.135124 0.137259 0.138807 0.134590 0.132454 0.134696 0.137900 0.141583 0.072874 0.081256 0.000000 0.072233 0.074636 0.068977 0.164860 0.136779 0.166142 H_st_T17858_pa 15 0.138113 0.134643 0.136085 0.137099 0.138273 0.138380 0.137473 0.137633 0.134643 0.132187 0.139768 0.140142 0.075703 0.078213 0.072233 0.000000 0.063905 0.063905 0.168224 0.132881 0.167583 H_st_A11597_pa 16 0.134109 0.129037 0.132508 0.134163 0.134483 0.134910 0.133255 0.134590 0.131280 0.129465 0.133842 0.140302 0.074796 0.076558 0.074636 0.063905 0.000000 0.065667 0.171267 0.12887